In [1]:
import tensorflow as tf
import os
import numpy as np
import cv2
import csv
from tqdm import tqdm

In [2]:
def parse_data(filename, label) :
    img_read = tf.io.read_file(filename)
    img_decode = tf.image.decode_jpeg(img_read, channels=3)
#     img = tf.image.resize(img_decode, [227,227])
    return img_decode, label

def create_dataset(filenames, labels, batch_size) :
    dataset = tf.data.Dataset.from_tensor_slices((filenames,labels))
    dataset = dataset.map(parse_data, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.shuffle(buffer_size=len(filenames))
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    
    return dataset

def read_labels(path) :
    img_classes = os.listdir(path)
    total_len = len(os.listdir(path+img_classes[0])) + len(os.listdir(path+img_classes[1])) + \
    len(os.listdir(path+img_classes[2]))+ len(os.listdir(path+img_classes[3])) + \
    len(os.listdir(path+img_classes[4]))+ len(os.listdir(path+img_classes[5]))+\
    len(os.listdir(path+img_classes[6]))

    names = []
    labels = np.zeros((total_len, len(img_classes)))
    j=0
    for i, img_class in enumerate(img_classes) :
        for img in os.listdir(os.path.join(path, img_class)) :
            names.append(os.path.join(path, img_class, img))
            labels[j][i] = 1
            j+=1
        
    return names, labels


batch_size = 16
images, labels = read_labels('./train/')
dataset = create_dataset(images, labels, batch_size)

In [5]:
from keras import Sequential, layers, Model

resnet50 = tf.keras.applications.resnet.ResNet50(include_top=False, weights='imagenet', input_shape=(227,227,3))

for i, layer in enumerate(resnet50.layers) :
    if i < 81 :
        layer.trainable = False
resnet50.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 227, 227, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 233, 233, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 114, 114, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 114, 114, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [6]:
model = Sequential()
model.add(resnet50)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(7, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 8, 8, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 7)                 14343     
Total params: 23,602,055
Trainable params: 22,098,951
Non-trainable params: 1,503,104
_________________________________________________________________


In [7]:
metrics = [
    tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.AUC(name='AUC')
]
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
             loss=tf.keras.losses.CategoricalCrossentropy(),
             metrics=metrics)

In [8]:
history = model.fit(dataset,
                    batch_size=16,
                    epochs=10)

Epoch 1/10
107/107 [==============================] - 397s 4s/step - loss: 1.2595 - accuracy: 0.6307 - precision: 0.7279 - recall: 0.4853 - AUC: 0.8882
Epoch 2/10
107/107 [==============================] - 405s 4s/step - loss: 0.5996 - accuracy: 0.8080 - precision: 0.8550 - recall: 0.7432 - AUC: 0.9677
Epoch 3/10
107/107 [==============================] - 395s 4s/step - loss: 0.3235 - accuracy: 0.8999 - precision: 0.9316 - recall: 0.8669 - AUC: 0.9896
Epoch 4/10
107/107 [==============================] - 425s 4s/step - loss: 0.2381 - accuracy: 0.9329 - precision: 0.9464 - recall: 0.9046 - AUC: 0.9937
Epoch 5/10
107/107 [==============================] - 422s 4s/step - loss: 0.1718 - accuracy: 0.9499 - precision: 0.9600 - recall: 0.9329 - AUC: 0.9963
Epoch 6/10
107/107 [==============================] - 406s 4s/step - loss: 0.1690 - accuracy: 0.9464 - precision: 0.9564 - recall: 0.9299 - AUC: 0.9964
Epoch 7/10
107/107 [==============================] - 434s 4s/step - loss: 0.1364 - accu

In [10]:
from tqdm import tqdm
model.save_weights('programmers_v3.ht')
test_path = './test/0'

with open('test_answer3.csv', 'w' , newline='') as f:
    wr = csv.writer(f)
    wr.writerow(['','answer value'])
    for i, image in tqdm(enumerate(os.listdir(test_path))) :
        img = cv2.imread(os.path.join(test_path, image))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, dsize=(227,227))
        img = tf.reshape(img, (-1, img.shape[0], img.shape[1], img.shape[2]))

        pred = model.predict(img)
        answer = np.argmax(pred[0])
        wr.writerow([i, answer])
    

350it [00:46,  7.50it/s]
